In [1]:
import pandas as pd
from functions_load_and_transform import thisWeek_DF, lastWeek_DF, thisWeek, lastWeek, team_list

In [2]:
def team_score_counter(row, teams_dict):
    # print(row["Winner"])
    if not row["Winner"] == "0":
        teams_dict[row["Home Team"]][3] +=1
        teams_dict[row["Away Team"]][3] +=1
        print(row["Winner"])
        if row["Winner"] == "Draw":
            teams_dict[row["Home Team"]][1] +=1
            teams_dict[row["Away Team"]][1] +=1
        else:
            if row["Winner"] == row["Home Team"]:
                teams_dict[row["Home Team"]][0] +=1
                teams_dict[row["Away Team"]][2] +=1
            else:
                teams_dict[row["Away Team"]][0] +=1
                teams_dict[row["Home Team"]][2] +=1
    return teams_dict

def get_team_scoring():
    df = pd.read_csv("data/results.csv")
    teams_dict = {}
    for team in team_list:
        teams_dict[team] = [0, 0, 0, 0] # Wins, Draws, Losses, Games Played
    df.apply(team_score_counter, teams_dict=teams_dict, axis=1)
    return teams_dict


In [3]:

# team_scores_DF

In [67]:
pd.read_csv("data/teams.csv")


,Wins,Draws,Losses,Games Played,Team,Division,Subdivision
0,1,0,0,1,Arizona Cardinals,NFC,West
1,1,0,0,1,Atlanta Falcons,NFC,South
2,1,0,0,1,Carolina Panthers,NFC,South
3,0,0,1,1,Chicago Bears,NFC,North
4,0,0,1,1,Dallas Cowboys,NFC,East
5,0,0,1,1,Detroit Lions,NFC,North
6,0,0,1,1,Green Bay Packers,NFC,North
7,1,0,0,1,Los Angeles Rams,NFC,West
8,1,0,0,1,Minnesota Vikings,NFC,North
9,0,0,1,1,New Orleans Saints,NFC,South


In [65]:
def calculate_w_d_l():
    team_scores_DF = pd.read_csv("data/teams.csv")
    results_DF = pd.read_csv("data/results.csv")

    count_draws = dict(zip(team_list, [0]*len(team_list)))
    draw_dict = results_DF.loc[results_DF["Winner"]=="Draw", ["Home Team", "Away Team"]].value_counts().to_dict()

    for team in draw_dict:
        count_draws[team[0]] += 1
        count_draws[team[1]] += 1

    for team in team_list:
        if team in list(results_DF.loc[results_DF["Winner"]==team, "Winner"]):
            team_scores_DF.loc[team_scores_DF["Team"]==team, "Wins"] = results_DF.loc[results_DF["Winner"]==team, "Winner"].value_counts()[0]
        if team in list(results_DF.loc[results_DF["Looser"]==team, "Looser"]):
            team_scores_DF.loc[team_scores_DF["Team"]==team, "Losses"] = results_DF.loc[results_DF["Looser"]==team, "Looser"].value_counts()[0]
        team_scores_DF.loc[team_scores_DF["Team"]==team, "Draws"] = count_draws[team]
    
    team_scores_DF["Games Played"] = team_scores_DF["Wins"]+team_scores_DF["Draws"]+team_scores_DF["Losses"]
    return team_scores_DF
calculate_w_d_l()

,Wins,Draws,Losses,Games Played,Team,Division,Subdivision
0,0,0,0,0,Arizona Cardinals,NFC,West
1,0,0,0,0,Atlanta Falcons,NFC,South
2,0,1,0,1,Carolina Panthers,NFC,South
3,0,0,0,0,Chicago Bears,NFC,North
4,0,0,0,0,Dallas Cowboys,NFC,East
5,0,0,0,0,Detroit Lions,NFC,North
6,0,0,0,0,Green Bay Packers,NFC,North
7,0,0,0,0,Los Angeles Rams,NFC,West
8,0,1,0,1,Minnesota Vikings,NFC,North
9,0,1,0,1,New Orleans Saints,NFC,South
